In [ ]:
# Using One-hot label

import torch
import numpy as np
import random
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import tensorflow as tf
import pandas as pd

# Define the data structure
class MyDataset(Dataset):
    def __init__(self,filepath):
        xy = np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len = xy.shape[0]
        
        # load the data after the second column of the dataset (RSS data)
        self.x_data = torch.from_numpy(xy[:,2:])
        # load the first two columns of the dataset (Coordinate)
        self.y_data = torch.from_numpy(xy[:,0:2])
        
        #print("The data has prepared...")
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

# Initialize the positioning error
error_0 = [[]for i in range(20)]
error_1 = [[]for i in range(20)]
error_2 = [[]for i in range(20)]
error_3 = [[]for i in range(20)]
error_4 = [[]for i in range(20)]
error_5 = [[]for i in range(20)]

realization = 500 # number of realizations
for z in range(realization):
    print(z)
    # Settings
    epochs = 200
    batch_size = 14
    lr = 0.008 #learning rate
    k=1
    
    # load traning and testing dataset
    train_file = "./dataset/03_14_train.csv"
    test_file = "./dataset/03_10_test.csv"
    mydataset_train = MyDataset(train_file)
    mydataset_test = MyDataset(test_file)
    train_loader = DataLoader(dataset=mydataset_train,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=2)
    test_loader = DataLoader(dataset=mydataset_test,
                             batch_size=10,
                             shuffle=False,
                             num_workers=2)

    # Model structure
    class AutoEncoder(nn.Module):
        def __init__(self):
            super(AutoEncoder, self).__init__()

            # Encoder
            self.encoder = nn.Sequential(
                nn.Linear(64, 48),
                nn.Tanh(),
                nn.Linear(48, 32),
                nn.Tanh(),
                nn.Linear(32, 16),
            )

            # Decoder
            self.decoder = nn.Sequential(
                nn.Linear(16, 32),
                nn.Tanh(),
                nn.Linear(32, 48),
                nn.Tanh(),
                nn.Linear(48, 64),
                nn.Tanh(),
            )

        def forward(self, inputs):
            codes = self.encoder(inputs)
            decoded = self.decoder(codes)
            return codes, decoded

    # Loss Function
    class My_loss(nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x, y):
            return torch.mean(torch.sqrt((torch.pow((30*x - 30*y), 2)).sum(axis=1))) # Average positioning error

    # Optimizer and loss function
    model = AutoEncoder()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.MSELoss()
    loss_function_locate = My_loss()

    # One-hot label setting
    density = np.zeros((3,13))
    density[1][6] = 1
    density = torch.from_numpy(density)

    # Add Label for training dataset
    for data, labels in train_loader:
        inputs = data.view(-1, 64) 
        locate = labels.view(-1,2)
    r = torch.tensor([2,16])
    j = torch.tensor([7,2])
    l_c = torch.floor((30*locate-r)/j).int()
    label_train = torch.ones((14,14))
    for i in range(locate.size(0)):
        label_train[i] = density[1-l_c[i][0]:3-l_c[i][0],6-l_c[i][1]:13-l_c[i][1]].flatten()

    # Train
    for epoch in range(epochs):
        for data, labels in train_loader:
            inputs = data.view(-1, 64) 
            locate = labels.view(-1,2)
            
            # Forward
            codes, decoded = model(inputs)
            # Backward
            optimizer.zero_grad()
            loss = loss_function(decoded, inputs) #the reconstruct error of AE
            loss.backward()
            optimizer.step()  
            
            for ep in range(10):
                codes, decoded = model(inputs)
                optimizer.zero_grad()
                # loss function for coordinate and Label estimation
                loss = loss_function_locate(codes[:,0:2], locate)+10*loss_function(codes[:,2:],label_train)
                loss.backward()
                optimizer.step()
        # Show progress
        locateloss = loss_function_locate(codes[:,0:2], locate)
        labelloss = loss_function(codes[:,2:], label_train)
        aeloss = loss_function(decoded, inputs)

        # Test
        if (epoch+1)%10 == 0 and epoch > 5:
            print(epoch)
            for data, labels in test_loader:
                inputs = data.view(-1, 64) 
                locate = labels.view(-1,2)
                codes, decoded = model(inputs)
                label_class = codes[:,2:]
                locate_test = codes[:,0:2]
                e = loss_function_locate(locate,codes[:,0:2])
            locate_0 = 30*codes[:,0:2] # estimated coordinates
            locate_estim_1 = torch.ones((10,2))
            locate_estim_2 = torch.ones((10,2))
            locate_estim_3 = torch.ones((10,2))
            locate_estim_4 = torch.ones((10,2))
            locate_estim_5 = torch.ones((10,2))
            
            # Fix out-of-range locations for easy labeling
            for i in range(locate.size(0)):
                if locate_0[i][0]<2: locate_0[i][0] = 2
                if locate_0[i][0]>=16: locate_0[i][0] = 16
                if locate_0[i][1]<16: locate_0[i][1] = 16
                if locate_0[i][1]>=30: locate_0[i][1] = 29.9

            # Estimate the grid where the point is lying
            label_test = torch.floor((locate_0-r)/j).int()

            # Select the grid center coordinates corresponding to the largest k values in the Label
            # Revise the estimated coordinates to obtain the revised coordinates
            r_1 = torch.ones((1,2));r_2 = torch.ones((1,2));r_3 = torch.ones((1,2));r_4 = torch.ones((1,2));r_5 = torch.ones((1,2))
            R_1 = torch.ones((1,2));R_2 = torch.ones((1,2));R_3 = torch.ones((1,2));R_4 = torch.ones((1,2));R_5 = torch.ones((1,2))
            for i in range(label_class.size(0)):
                index_0 = label_test[i][0]*7+label_test[i][1]
                index_1 = torch.argmax(label_class[i],0)
                r_1[0][0] = (index_1/7).int()
                r_1[0][1] = index_1%7
                R_1[0][0] = 5.5+7*r_1[0][0]
                R_1[0][1] = 17+2*r_1[0][1]
                label_class[i][index_1] = -1
                index_2 = torch.argmax(label_class[i],0)
                r_2[0][0] = (index_2/7).int()
                r_2[0][1] = index_2%7
                R_2[0][0] = 5.5+7*r_2[0][0]
                R_2[0][1] = 17+2*r_2[0][1]
                label_class[i][index_2] = -1
                index_3 = torch.argmax(label_class[i],0)
                r_3[0][0] = (index_3/7).int()
                r_3[0][1] = index_3%7
                R_3[0][0] = 5.5+7*r_3[0][0]
                R_3[0][1] = 17+2*r_3[0][1]
                label_class[i][index_3] = -1
                index_4 = torch.argmax(label_class[i],0)
                r_4[0][0] = (index_4/7).int()
                r_4[0][1] = index_4%7
                R_4[0][0] = 5.5+7*r_4[0][0]
                R_4[0][1] = 17+2*r_4[0][1]
                label_class[i][index_4] = -1
                index_5 = torch.argmax(label_class[i],0)
                r_5[0][0] = (index_5/7).int()
                r_5[0][1] = index_5%7
                R_5[0][0] = 5.5+7*r_5[0][0]
                R_5[0][1] = 17+2*r_5[0][1]
                R_one = R_1
                R_two = (R_1+R_2)/2
                R_three = (R_1+R_2+R_3)/3
                R_four = (R_1+R_2+R_3+R_4)/4
                R_five = (R_1+R_2+R_3+R_4+R_5)/5
                # the revised coordinates
                locate_estim_1[i] = (R_one+locate_0[i])/2
                locate_estim_2[i] = (R_two+locate_0[i])/2
                locate_estim_3[i] = (R_three+locate_0[i])/2
                locate_estim_4[i] = (R_four+locate_0[i])/2
                locate_estim_5[i] = (R_five+locate_0[i])/2
            # Positioning error
            error_0[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_0/30).detach().numpy())
            error_1[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_estim_1/30).detach().numpy())
            error_2[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_estim_2/30).detach().numpy())
            error_3[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_estim_3/30).detach().numpy())
            error_4[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_estim_4/30).detach().numpy())
            error_5[int((epoch+1)/10-1)].append(loss_function_locate(locate,locate_estim_5/30).detach().numpy())

# Save the result
for i in range(20):
    outcome_file = "./outcome/our_model_update_110_01label_"+str(10+i*10)+".csv"  
    error = [error_0[i],error_1[i],error_2[i],error_3[i],error_4[i],error_5[i]]
    error = np.array(error)
    data1 = pd.DataFrame(error.T)
    data1.to_csv(outcome_file,header=None,index=None)